Effectively representing temporal dynamics in large datasets requires selecting appropriate visualization techniques that ensure responsiveness while providing both a macroscopic view of overall trends and a microscopic view of fine details. This guide will explore various methods, such as **WebGL Rendering**, **LTTB Downsampling**, **Datashader Rasterizing**, and **Minimap**, each suited for different aspects of large timeseries data visualization. We predominantly demonstrate the use of hvPlot syntax, leveraging HoloViews for more complex requirements. Although hvPlot supports multiple backends, including Matplotlib and Plotly, our focus will be on Bokeh due to its advanced capabilities in handling large timeseries data.


## Getting the data 

Here we have a DataFrame with 1.2 million rows containing standardized data from 5 different sensors.

In [ ]:
import pandas as pd

df = pd.read_parquet("https://datasets.holoviz.org/sensor/v1/data.parq")
df

In [ ]:
df0 = df[df.sensor=='0']

Let's go ahead and plot this data using various approaches.

## WebGL Rendering

### Canvas Rendering - Prior Default
Rendering in hvPlot and HoloViews for Bokeh plots has evolved significantly. Prior to HoloViews version 1.16.0 (May 2023), Bokeh's custom HTML **Canvas** rendering was the default. This approach works well for datasets up to a few tens of thousands of points but struggles above 100K points, particularly in terms of zooming and panning speed. These days, if you want to utilize Canvas rendering, use `import holoviews as hv; hv.renderer("bokeh").webgl = False` prior to creating your hvPlot or HoloViews object.

### WebGL Rendering - Current Default
Around mid-2023, the adoption of improved **WebGL** as the default for hvPlot and HoloViews allowed for smoother interactions with larger datasets by utilizing GPU-acceleration. It's important to note that WebGL performance can vary based on your machine's specifications. For example, some Mac models may not exhibit a marked improvement in WebGL performance over Canvas due to GPU specifications.

In [ ]:
import holoviews as hv; hv.extension('bokeh')
import hvplot.pandas  # noqa: F401

df0.hvplot(x="time", y="value", responsive=True, min_height=300, autorange='y', title="WebGL")

<div class="alert alert-info">

<b>Note:</b> `autorange='y'` is used here for automatic y-axis scaling, a feature from HoloViews 1.17 and hvPlot 0.9.0. You can omit that option if you prefer to set the y scaling manually using the zoom tool.

</div>

Alone, both Canvas and WebGL rendering have a common limitation: they transfer the entire dataset from the server to the browser. This can be a significant bottleneck, especially for remote server setups or datasets larger than a million points. To address this, we'll explore other techniques like LTTB Downsampling, which focus on delivering only the necessary data for the current view. These methods offer more scalable solutions for interacting with large timeseries data, as we'll see in the following sections.

## LTTB Downsampling

### The Challenge with Simple Downsampling

A straightforward approach to handling large datasets might involve plotting every _n_th datapoint using a method like df.sample:

In [ ]:
df0.sample(500).hvplot(x="time", y="value", responsive=True, min_height=300, autorange='y',
                       title = "Decimation: Don't do this!")

However, this method, known as decimation or arbitrarily strided sampling, can lead to [aliasing](https://en.wikipedia.org/wiki/Downsampling_(signal_processing)), where the resulting plot misrepresents the actual data by missing crucial peaks, troughs, or slopes. For instance, significant variations visible in the WebGL plot of the previous section might be entirely absent in a decimated plot, making this approach generally inadvisable for accurate data representation.

### The LTTB Solution

To address this, a more sophisticated method like the [Largest Triangle Three Buckets (LTTB)](https://skemman.is/handle/1946/15343) algorithm can be employed. LTTB allows data points not contributing significantly to the visible shape to be dropped, reducing the amount of data to send to the browser but preserving the appearance (and particularly the envelope, i.e. highest and lowest values in a region).

In hvPlot, adding `downsample=True` will enable the LTTB algorithm, which will automatically choose an appropriate number of samples for the current plot:

In [ ]:
df0.hvplot(x="time", y="value", downsample=True, responsive=True, min_height=300, autorange='y', title="LTTB")

The LTTB plot will closely resemble the WebGL plot in appearance, but in general, it is rendered much more quickly (especially for local browsing of remote computation). As LTTB depends on Python dynamically, if you are running this notebook locally with a live Python process, the plot will automatically update with additional detail as you zoom in.

With LTTB, it is now practical to include all of the different sensors in a single plot without slowdown, updating to show more detail when zooming in: 

In [ ]:
df.hvplot(x="time", y="value", downsample=True, by='sensor', responsive=True, min_height=300, title="LTTB By Sensor")

This makes LTTB an ideal default method for exploring timeseries datasets, particularly when the dataset size is unknown or too large for standard WebGL rendering.

## Datashader Rasterizing

In [ ]:
from holoviews.operation.resample import ResampleOperation2D
ResampleOperation2D.width=1200
ResampleOperation2D.height=500

<div class="alert alert-info">

<b>Note:</b> This code above sets the default image size for Datashader renderings. It ensures images appear at high resolution when the notebook is displayed on our website, addressing the absence of dynamic resizing in this context. For interactive or local use, this adjustment is not critical.

</div>




Bokeh WebGL and LTTB both send data to the web browser and ask the web browser to "connect the dots" between them by drawing a line in the browser page, with LTTB simply sending fewer points. [Datashader](https://datashader.org) works in a different way, rendering the data into a frame buffer on the server, and then sending that buffer to the web browser rather than the individual data points. Thus Datashader will send only a fixed amount of data (the rendered plot), potentially greatly speeding up plots of the largest datasets. As for LTTB, plots will only be updated after a zoom or pan if Python is still running, because Python is what renders and supplies the updated image. Setting the argument `line_width` to a value above 0 will enable [anti-aliasing](https://en.wikipedia.org/wiki/Anti-aliasing) of the line. 

In [ ]:
df0.hvplot(x="time", y="value", rasterize=True, cnorm='eq_hist', padding=(0, 0.1), line_width=1,
           responsive=True, min_height=300, autorange='y', title="Rasterize")

If you zoom in enough, you'll see a normal line, but for a long timeseries in a zoomed out plot like this one, what you will see is Datashader's "aggregation" of *all* the line segments between the points, with darker colors indicating areas where the data trace goes back and forth multiple times in a single pixel (with the number of "switchbacks" indicated in the color key). This representation conveys a lot more about the behavior of this data, with the previous plots showing a single solid color regardless of how many line segments crossed that pixel. Datashader rendering can be used to get a good overview of the full shape of a long timeseries, helping you understand how the signal varies even when the steps involved are smaller than the pixels on the screen.

For data with different "categories" (sensors, in this case), Datashader can assign a different color to each of the sensor categories and then aggregating all of them into the final display by mixing their colors:

In [ ]:
df.hvplot(x="time", y="value", datashade=True, hover=True, padding=(0, 0.1), responsive=True,
          min_height=300, autorange='y', line_width=1, by='sensor', title="Rasterize categories")

This categorical color mixing can help indicate when traces overlap each other, to give you a clue when to zoom in, and becomes particularly important the more categories there are.

[The example above needs `rasterize`, plus instant inspection. Also needs to illustrate what happens when very large numbers of traces overlap.] 

## Minimap

The LTTB and Datashader options are about rendering or omitting datapoints when showing a large time range that would include many data points. What if you have years of data, but the timescale involved is such that you typically study a single day or a single hour? In that case the new "minimap" approach can help you ensure that you see the larger context while actually plotting only the smaller time range.

A minimap is added using the HoloViews RangeToolLink:

In [ ]:
from holoviews.plotting.links import RangeToolLink

# Does not yet work with downsample1d. For now, to make it easier on the browser, let's just take a subset of the data
downsampled_df = df.iloc[::10]

plot    = df0.hvplot(x="time", y="value", height=500)
minimap = df0.hvplot(x="time", y="value", height=150).opts(ylabel='', xlabel='')

link = RangeToolLink(minimap, plot, axes=["x", "y"], boundsx=(None, pd.Timestamp("2022-02-01")), boundsy=(-5, 5))

(plot + minimap).opts(shared_axes=False).cols(1)

Here, you can drag the grey box on the bottom plot and the top plot will update to show that range of the data, letting you explore a large dataset while plotting only a short stretch at a time.